# BIP's error function

To the error function does expected to receive as input the following pandas.DataFrame struncture:

| StoreID  | Month | Region | NumberOfSales | _NumberOfSales |
| ---------|----------------|---------------| ---------------|
| 1000	   |3      | 4      | 16            | 16             |
| 1000	   |4      | 4      | 30            | 23             |
| 1001	   |3      | 6      | 410           | 411            |
| 1001	   |4      | 27     | 3130          | 3120           |
| 1002	   |3      | 58     | 10            | 8              |

Where:
 
 - *NumberOfSales* are the **test actual values**
 - *_NumberOfSales* are **the predicted values**
 
 

Start from test set to simulate a predicted dataset

In [1]:
from import_man import *

df = pd.read_csv('./dataset/preprocessed_train.csv')

print("Shape before: " + str(df.shape))

# Let's work on a reduced instance of the test set
df = df.sample(n=5000)

print("Shape after: " + str(df.shape))

Shape before: (523021, 51)
Shape after: (5000, 51)


In [2]:
error_evaluation_columns = ['StoreID', 'D_Month', 'Region', 'NumberOfSales', '_NumberOfSales']

# Create fake predicted sales 
df['_NumberOfSales'] = df.NumberOfSales.apply(lambda x: df['NumberOfSales'].sample().values[0])


# Save a copy of the dataframe to be passed to the function in order to evaluate it
df_BIP_err_fun_eval = df


# Remove useless columns and select all those required.
# Implicit check that all the required attributes to compute the error are present.
df = df[error_evaluation_columns]

df.head(20)

,StoreID,D_Month,Region,NumberOfSales,_NumberOfSales
211030,1301,5,2,5045.000000,1426.000000
371473,1532,3,9,3245.000000,4865.000000
201622,1287,1,2,6701.000000,4032.224215
124833,1179,4,2,4919.000000,5906.613599
307549,1440,4,6,4389.000000,6456.000000
6151,1008,7,9,5107.000000,4183.000000
505075,1723,12,9,5080.000000,3469.000000
36236,1051,2,2,4616.000000,5189.000000
208360,1297,1,9,6031.000000,5095.000000
33447,1047,6,9,5695.000000,4639.000000


In [3]:
# let's keep only march and april
# df = df.loc[df['D_Month'].isin([3, 4])] No. let's just evaluate months provided

# sum everything keeping distinguished: Month, Store and Region
df_sum_by_month = df.groupby(['D_Month', 'StoreID', 'Region']).sum()
df_sum_by_month.head()

NumberOfSales  _NumberOfSales
D_Month StoreID Region                               
1       1000    7         7675.446488     5468.000000
        1002    3         5892.000000     6886.000000
        1003    7         6444.000000     3583.955075
        1006    10       13113.342669    11211.054366
        1012    4        20747.000000    15604.000000

In [4]:
# compute the difference between actual and predicted NumberOfSales and do the abs
df_sum_by_month['abs_diff'] = df_sum_by_month.apply(lambda x: abs(x['NumberOfSales'] - x['_NumberOfSales']), axis=1)
df_sum_by_month.head()

NumberOfSales  _NumberOfSales     abs_diff
D_Month StoreID Region                                            
1       1000    7         7675.446488     5468.000000  2207.446488
        1002    3         5892.000000     6886.000000   994.000000
        1003    7         6444.000000     3583.955075  2860.044925
        1006    10       13113.342669    11211.054366  1902.288303
        1012    4        20747.000000    15604.000000  5143.000000

In [5]:
# Let's sum over the region
df_sums_by_region = df_sum_by_month.groupby(['Region']).sum()
df_sums_by_region.head(20)

,NumberOfSales,_NumberOfSales,abs_diff
Region,,,
0,1.839191e+06,1.803487e+06,7.310294e+05
1,7.913249e+05,8.083676e+05,3.996380e+05
2,3.089159e+06,3.198266e+06,1.247008e+06
3,3.725963e+06,3.250695e+06,1.632221e+06
4,6.570828e+05,5.923866e+05,2.801355e+05
5,1.595216e+06,1.774465e+06,7.822894e+05
6,1.074973e+06,1.229042e+06,4.717288e+05
7,2.100843e+06,2.248096e+06,9.059407e+05
8,8.030780e+05,9.191164e+05,3.257268e+05


In [6]:
# Divide the difference between actual and predicted NumberOfSales by the sum of actual
df_sums_by_region['E_r'] = df_sums_by_region['abs_diff'] / df_sums_by_region['NumberOfSales']

df_sums_by_region.head(20)

,NumberOfSales,_NumberOfSales,abs_diff,E_r
Region,,,,
0,1.839191e+06,1.803487e+06,7.310294e+05,0.397473
1,7.913249e+05,8.083676e+05,3.996380e+05,0.505024
2,3.089159e+06,3.198266e+06,1.247008e+06,0.403672
3,3.725963e+06,3.250695e+06,1.632221e+06,0.438067
4,6.570828e+05,5.923866e+05,2.801355e+05,0.426332
5,1.595216e+06,1.774465e+06,7.822894e+05,0.490397
6,1.074973e+06,1.229042e+06,4.717288e+05,0.438828
7,2.100843e+06,2.248096e+06,9.059407e+05,0.431227
8,8.030780e+05,9.191164e+05,3.257268e+05,0.405598


In [7]:
# Get the number of regions
N_regions = len(df.Region.unique())

print("Number of regions: {}".format(N_regions))

Number of regions: 11


In [8]:
step_by_step_error = df_sums_by_region['E_r'].sum() / N_regions

print("BIP error: {}".format(step_by_step_error))

BIP error: 0.43009487751872366


### Test of the BIP error function implemented

Test that the get_BIP_error function works as expected

In [9]:
from BIP_error import get_BIP_error

df_BIP_err_fun_eval[error_evaluation_columns].head()

,StoreID,D_Month,Region,NumberOfSales,_NumberOfSales
211030,1301,5,2,5045.0,1426.000000
371473,1532,3,9,3245.0,4865.000000
201622,1287,1,2,6701.0,4032.224215
124833,1179,4,2,4919.0,5906.613599
307549,1440,4,6,4389.0,6456.000000


In [10]:
# use the function
error_from_BIP = get_BIP_error(df_BIP_err_fun_eval)

# let's assert that the two errors are equal
np.testing.assert_almost_equal(step_by_step_error, error_from_BIP, decimal=10)

Number of regions identified: 11
BIP total error: 0.43009487751872366


### Test of the BIP error function: case of predictions = 0

Let's test how the function behaves if the predictions are all equal to 0.

In [11]:
test_zero = df_BIP_err_fun_eval
test_zero['_NumberOfSales'] = 0
test_zero[error_evaluation_columns].head()

,StoreID,D_Month,Region,NumberOfSales,_NumberOfSales
211030,1301,5,2,5045.0,0
371473,1532,3,9,3245.0,0
201622,1287,1,2,6701.0,0
124833,1179,4,2,4919.0,0
307549,1440,4,6,4389.0,0


In [12]:
# which is the erro in this case? expect 1.00
error_zero = get_BIP_error(test_zero)

Number of regions identified: 11
BIP total error: 1.0


### Test of the BIP error function: case of predictions = predictions + n

Let's test what happen if number of sales are incremented of a number *n*.

In [13]:
n= 100
test_n = df_BIP_err_fun_eval
test_n['_NumberOfSales'] = test_n['NumberOfSales'] + n
test_n[error_evaluation_columns].head()

,StoreID,D_Month,Region,NumberOfSales,_NumberOfSales
211030,1301,5,2,5045.0,5145.0
371473,1532,3,9,3245.0,3345.0
201622,1287,1,2,6701.0,6801.0
124833,1179,4,2,4919.0,5019.0
307549,1440,4,6,4389.0,4489.0


In [14]:
# which is the erro in this case?
error_n = get_BIP_error(test_n)

Number of regions identified: 11
BIP total error: 0.02083719471050353
